In [1]:
import os
import gc
import numpy as np
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer
from scipy.stats import rankdata
import textstat
from tqdm import tqdm
import scml
import mylib

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda
NVIDIA GeForce GTX 1060 6GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [3]:
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()

In [4]:
%%time
df = pd.read_csv("input/validation_data.csv", engine="c", low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30108 entries, 0 to 30107
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   worker      30108 non-null  int64 
 1   less_toxic  30108 non-null  object
 2   more_toxic  30108 non-null  object
dtypes: int64(1), object(2)
memory usage: 705.8+ KB
Wall time: 210 ms


In [5]:
texts = set(df["less_toxic"]) | set(df["more_toxic"])
texts = list(texts)
df = pd.DataFrame(data={"text": texts})
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    14251 non-null  object
dtypes: object(1)
memory usage: 111.5+ KB


# Preprocess Text

In [6]:
def preprocess(row) -> str:
    return mylib.preprocess(row["text"])


col = "ptext"
df[col] = df.progress_apply(preprocess, axis=1)

100%|███████████████████████████████████████| 14251/14251 [00:11<00:00, 1292.06it/s]


# Character level features

In [7]:
%%time
col = "length"
df[col] = df["ptext"].str.len()
df[col] = df[col].astype(np.int16)

Wall time: 9 ms


In [8]:
def digit_frac(row) -> float:
    return mylib.digit_frac(row["ptext"])


def letter_frac(row) -> float:
    return mylib.letter_frac(row["ptext"])


def space_frac(row) -> float:
    return mylib.space_frac(row["ptext"])


def punc_frac(row) -> float:
    return mylib.punc_frac(row["ptext"])


def upper_frac(row) -> float:
    return mylib.upper_frac(row["ptext"])

In [9]:
col = "digit_frac"
df[col] = df.progress_apply(digit_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 24300.25it/s]


In [10]:
col = "letter_frac"
df[col] = df.progress_apply(letter_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 22001.07it/s]


In [11]:
col = "space_frac"
df[col] = df.progress_apply(space_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 23391.29it/s]


In [12]:
col = "punc_frac"
df[col] = df.progress_apply(punc_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 21204.85it/s]


In [13]:
col = "upper_frac"
df[col] = df.progress_apply(upper_frac, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 23935.19it/s]


# Textstat features

In [14]:
def syllable_count(row) -> int:
    return textstat.syllable_count(row["ptext"])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row["ptext"])


def sentence_count(row) -> int:
    return textstat.sentence_count(row["ptext"])


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row["ptext"])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row["ptext"])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row["ptext"])


def smog_index(row) -> float:
    return textstat.smog_index(row["ptext"])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row["ptext"])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row["ptext"])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row["ptext"])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row["ptext"])

In [15]:
col = "flesch_reading_ease"
df[col] = df.progress_apply(flesch_reading_ease, axis=1)
df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████| 14251/14251 [00:03<00:00, 3688.99it/s]


In [16]:
col = "flesch_kincaid_grade"
df[col] = df.progress_apply(flesch_kincaid_grade, axis=1)
df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████| 14251/14251 [00:02<00:00, 7018.25it/s]


In [17]:
col = "syllable_count"
df[col] = df.progress_apply(syllable_count, axis=1)
df[col] = df[col].astype(np.int16)

100%|██████████████████████████████████████| 14251/14251 [00:01<00:00, 10799.59it/s]


In [18]:
col = "lexicon_count"
df[col] = df.progress_apply(lexicon_count, axis=1)
df[col] = df[col].astype(np.int16)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 47503.53it/s]


In [19]:
col = "sentence_count"
df[col] = df.progress_apply(sentence_count, axis=1)
df[col] = df[col].astype(np.int16)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 26516.12it/s]


In [20]:
col = "gunning_fog"
df[col] = df.progress_apply(gunning_fog, axis=1)
df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████| 14251/14251 [00:02<00:00, 5450.53it/s]


In [21]:
col = "smog_index"
df[col] = df.progress_apply(smog_index, axis=1)
df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████| 14251/14251 [00:01<00:00, 7310.24it/s]


In [22]:
col = "automated_readability_index"
df[col] = df.progress_apply(automated_readability_index, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████████████████| 14251/14251 [00:00<00:00, 17452.53it/s]


In [23]:
col = "coleman_liau_index"
df[col] = df.progress_apply(coleman_liau_index, axis=1)
df[col] = df[col].astype(np.float32)

100%|██████████████████████████████████████| 14251/14251 [00:01<00:00, 14000.79it/s]


In [24]:
col = "linsear_write_formula"
df[col] = df.progress_apply(linsear_write_formula, axis=1)
df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████| 14251/14251 [00:01<00:00, 8102.24it/s]


In [25]:
col = "dale_chall_readability_score"
df[col] = df.progress_apply(dale_chall_readability_score, axis=1)
df[col] = df[col].astype(np.float32)

100%|███████████████████████████████████████| 14251/14251 [00:02<00:00, 5693.00it/s]


# Detoxify labels

In [26]:
sentences = list(df["ptext"])
batch_size = 64
model_max_length = 512
dtfy_fs = []

In [27]:
%%time
prefix = "dto_"
res = mylib.detoxify_labels(
    sentences,
    checkpoint="pretrained/unitaryai/detoxify/toxic_original-c1212f89.ckpt",
    config_dir="pretrained/bert-base-uncased",
    model_max_length=model_max_length,
    device=device,
    batch_size=batch_size
)
for k, v in res.items():
    col = prefix + k
    df[col] = v
    df[col] = df[col].astype(np.float32)
    dtfy_fs.append(col)
gc.collect()

Wall time: 10min 30s


0

In [28]:
%%time
prefix = "dtu_"
res = mylib.detoxify_labels(
    sentences,
    checkpoint="pretrained/unitaryai/detoxify/toxic_debiased-c7548aa0.ckpt",
    config_dir="pretrained/roberta-base",
    model_max_length=model_max_length,
    device=device,
    batch_size=batch_size
)
for k, v in res.items():
    col = prefix + k
    df[col] = v
    df[col] = df[col].astype(np.float32)
    dtfy_fs.append(col)
gc.collect()

Wall time: 10min 11s


4

In [29]:
%%time
prefix = "dtm_"
res = mylib.detoxify_labels(
    sentences,
    checkpoint="pretrained/unitaryai/detoxify/multilingual_debiased-0b549669.ckpt",
    config_dir="pretrained/xlm-roberta-base",
    model_max_length=model_max_length,
    device=device,
    batch_size=batch_size
)
for k, v in res.items():
    col = prefix + k
    df[col] = v
    df[col] = df[col].astype(np.float32)
    dtfy_fs.append(col)
gc.collect()

Wall time: 10min 11s


422

In [30]:
print(dtfy_fs)

['dto_toxicity', 'dto_severe_toxicity', 'dto_obscene', 'dto_threat', 'dto_insult', 'dto_identity_attack', 'dtu_toxicity', 'dtu_severe_toxicity', 'dtu_obscene', 'dtu_identity_attack', 'dtu_insult', 'dtu_threat', 'dtu_sexual_explicit', 'dtm_toxicity', 'dtm_severe_toxicity', 'dtm_obscene', 'dtm_identity_attack', 'dtm_insult', 'dtm_threat', 'dtm_sexual_explicit']


# Embeddings

In [31]:
model = SentenceTransformer("pretrained/sentence-transformers/paraphrase-MiniLM-L6-v2", device=device)
model.max_seq_length = 128
em = model.encode(sentences=sentences, batch_size=1000, show_progress_bar=True, convert_to_numpy=True)
print(f"em.shape={em.shape}")

Batches:   0%|          | 0/15 [00:00<?, ?it/s]

em.shape=(14251, 384)


In [32]:
%%time
em_size = em.shape[1]
em_cols = [f"zz{i:04d}" for i in range(em_size)]
df[em_cols] = em
df[em_cols] = df[em_cols].astype(np.float32)
del sentences

s:\dev\seahrh\kaggle-jigsaw-toxic-severity-rating\env\lib\site-packages\pandas\core\frame.py:3673: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self[col] = igetitem(value, i)


Wall time: 283 ms


# Review data

In [33]:
char_fs = ["length", "digit_frac", "letter_frac", "space_frac", "punc_frac", "upper_frac"]
textstat_fs = ["syllable_count", "lexicon_count", "sentence_count", "flesch_reading_ease", 
           "flesch_kincaid_grade", "gunning_fog", "smog_index", "automated_readability_index", 
           "coleman_liau_index", "linsear_write_formula", "dale_chall_readability_score"]
cols = char_fs + textstat_fs + dtfy_fs
df[cols].describe(percentiles=percentiles)

,length,digit_frac,letter_frac,space_frac,punc_frac,upper_frac,syllable_count,lexicon_count,sentence_count,flesch_reading_ease,flesch_kincaid_grade,gunning_fog,smog_index,automated_readability_index,coleman_liau_index,linsear_write_formula,dale_chall_readability_score,dto_toxicity,dto_severe_toxicity,dto_obscene,dto_threat,dto_insult,dto_identity_attack,dtu_toxicity,dtu_severe_toxicity,dtu_obscene,dtu_identity_attack,dtu_insult,dtu_threat,dtu_sexual_explicit,dtm_toxicity,dtm_severe_toxicity,dtm_obscene,dtm_identity_attack,dtm_insult,dtm_threat,dtm_sexual_explicit
count,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000,14251.000000
mean,408.604238,0.009755,0.772961,0.173892,0.043392,0.086143,100.923233,71.034945,3.974388,63.909294,9.625759,11.228957,4.309607,13.042585,9.093493,9.910800,9.519163,0.458109,0.067606,0.294862,0.028744,0.277615,0.061937,0.533886,0.046184,0.297274,0.061978,0.305610,0.018006,0.098901,0.509862,0.062107,0.259716,0.056330,0.269497,0.026773,0.135149
std,688.809292,0.032184,0.056109,0.024900,0.043562,0.176861,169.861304,117.409068,8.581545,315.695831,48.136086,21.454451,5.286682,90.704788,97.974510,9.309810,3.388893,0.427506,0.155756,0.393250,0.123391,0.368762,0.177876,0.398096,0.120675,0.390690,0.161861,0.360564,0.095459,0.217916,0.413721,0.158119,0.371604,0.162481,0.351557,0.111058,0.273091
min,8.000000,0.000000,0.004427,0.000403,0.000000,0.000000,2.000000,2.000000,1.000000,-36681.820312,-3.100000,0.800000,0.000000,-9.300000,-14.150000,0.000000,0.100000,0.000535,0.000079,0.000149,0.000085,0.000164,0.000127,0.000352,0.000001,0.000017,0.000060,0.000061,0.000016,0.000010,0.000126,0.000011,0.000052,0.000071,0.000128,0.000017,0.000013
1%,22.000000,0.000000,0.549739,0.094467,0.000000,0.000000,5.000000,4.000000,1.000000,-69.965000,-1.900000,1.600000,0.000000,-2.900000,-2.950000,1.000000,1.105000,0.000662,0.000087,0.000165,0.000099,0.000175,0.000136,0.001290,0.000003,0.000073,0.000136,0.000104,0.000034,0.000036,0.000405,0.000016,0.000085,0.000108,0.000181,0.000027,0.000018
5%,31.000000,0.000000,0.683673,0.131579,0.000000,0.000000,8.000000,6.000000,1.000000,30.200001,0.500000,2.400000,0.000000,0.300000,0.450000,2.000000,6.340000,0.000958,0.000093,0.000179,0.000109,0.000186,0.000145,0.005784,0.000007,0.000271,0.000345,0.000297,0.000067,0.000100,0.001091,0.000024,0.000150,0.000175,0.000374,0.000038,0.000025
10%,44.000000,0.000000,0.722973,0.145038,0.012579,0.008547,11.000000,8.000000,1.000000,43.560001,1.900000,3.200000,0.000000,2.000000,2.370000,2.500000,6.930000,0.001849,0.000100,0.000211,0.000119,0.000219,0.000164,0.016431,0.000013,0.000534,0.000588,0.000892,0.000100,0.000186,0.003285,0.000036,0.000332,0.000293,0.000899,0.000059,0.000039
20%,69.000000,0.000000,0.750000,0.158537,0.020170,0.016471,17.000000,12.000000,1.000000,55.610001,3.500000,5.200000,0.000000,4.100000,4.290000,4.000000,7.640000,0.009245,0.000114,0.000473,0.000164,0.000519,0.000260,0.060106,0.000030,0.001509,0.001155,0.003852,0.000177,0.000442,0.021863,0.000106,0.001394,0.000720,0.003930,0.000166,0.000138
30%,99.000000,0.000000,0.765060,0.166031,0.025000,0.021622,24.000000,18.000000,1.000000,62.680000,4.600000,6.730000,0.000000,5.600000,5.510000,5.000000,8.140000,0.029829,0.000154,0.001079,0.000285,0.001395,0.000481,0.154698,0.000063,0.004130,0.002022,0.012476,0.000273,0.000932,0.089935,0.000281,0.003921,0.001467,0.010699,0.000392,0.000394
40%,138.000000,0.000000,0.775180,0.171315,0.029557,0.026316,34.000000,24.000000,2.000000,68.099998,5.700000,8.040000,0.000000,6.900000,6.490000,6.200000,8.570000,0.092410,0.000305,0.003357,0.000498,0.

In [34]:
cols = ["text"] + cols + em_cols
df[cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14251 entries, 0 to 14250
Data columns (total 422 columns):
 #    Column                        Non-Null Count  Dtype  
---   ------                        --------------  -----  
 0    text                          14251 non-null  object 
 1    length                        14251 non-null  int16  
 2    digit_frac                    14251 non-null  float32
 3    letter_frac                   14251 non-null  float32
 4    space_frac                    14251 non-null  float32
 5    punc_frac                     14251 non-null  float32
 6    upper_frac                    14251 non-null  float32
 7    syllable_count                14251 non-null  int16  
 8    lexicon_count                 14251 non-null  int16  
 9    sentence_count                14251 non-null  int16  
 10   flesch_reading_ease           14251 non-null  float32
 11   flesch_kincaid_grade          14251 non-null  float32
 12   gunning_fog                   14251 non-null

In [35]:
%%time
df[cols].to_parquet("output/val.parquet", index=False)

Wall time: 491 ms
